This notebook is a conceptual research prototype. References to ‘AGI’ or ‘superintelligence’ describe aspirational goals and do not indicate the presence of real general intelligence. Use at your own risk.

# 👁️‍🗨️ Alpha-Factory v1 – Cross-Industry AGENTIC α-AGI  
## _(Google Colab, self-contained)_

**What you get in < 10 minutes**

* 🪄 Five domain agents (finance, biotech, climate, mfg, policy) orchestrated by `backend/orchestrator.py`
* 🏋 Continual PPO trainer self-improves agents
* 🔐 Guard-rails (MCP), ed25519 prompt signing, policy deny-list
* 📊 Prometheus **9090** & Grafana **3000** exposed through public *ngrok* URLs
* ⚡ 30-second k6 load-test to prove antifragility
* 📴 Works offline (Mixtral-8×7B GGML) or online (OpenAI key)

> **Paste an `OPENAI_API_KEY` below for better quality,** or leave blank to run fully offline.
The script will also generate a random `API_TOKEN` in `.env` for authenticated requests.

In [ ]:
OPENAI_API_KEY = ""  # ← 🔑  put your key here or leave empty

### Optional offline mode

Download a `.gguf` weight and set `LLAMA_MODEL_PATH` before running the setup cell. `llama-cpp-python` or `ctransformers` must be installed for local inference:

```bash
pip install openai-agents llama-cpp-python ctransformers
```

Example weights and rough CPU throughput:

| Model | Size | ~tokens/s |
|-------|------|-----------|
| TinyLlama-1.1B-Chat Q4_K_M | 380 MB | ~20 |
| Llama-3-8B-Instruct Q4_K_M | 4 GB | ~5 |
| Mixtral-8x7B-Instruct Q4_0 | 7 GB | ~3 |

When `LLAMA_MODEL_PATH` is defined the orchestrator loads the weight instead of calling OpenAI.


In [ ]:
%%bashif [ ! -d AGI-Alpha-Agent-v0 ]; then  git clone --depth 1 https://github.com/MontrealAI/AGI-Alpha-Agent-v0.gitficd AGI-Alpha-Agent-v0pip install -q -r alpha_factory_v1/requirements-colab.txt

### Offline wheelhouse install
Mount a Google Drive folder (or local path) containing pre-built wheels and install packages with `pip --no-index --find-links <wheelhouse>`. The definitive package list is in `alpha_factory_v1/requirements-colab.lock`.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
wheelhouse = '/content/drive/MyDrive/wheelhouse'  # or any local path
!pip install --no-index --find-links $wheelhouse -r AGI-Alpha-Agent-v0/alpha_factory_v1/requirements-colab.lock


## ⚙️ Patch orchestrator: add `/update_model` hot-reload endpoint

In [ ]:
%%bash
apply_patch() {
python - <<'PY'
import importlib, textwrap, pathlib, inspect
src = pathlib.Path('AGI-Alpha-Agent-v0/alpha_factory_v1/backend/orchestrator.py')
code = src.read_text()
if 'update_model' in code:
    print('✅ update_model already present'); raise SystemExit
insertion = textwrap.dedent('''\n@router.post("/agent/{agent_id}/update_model")\nasync def update_model(agent_id: str, file: bytes = File(...)):\n    if agent_id not in AGENT_REGISTRY:\n        raise HTTPException(status_code=404, detail="agent not found")\n    import tempfile, zipfile, io\n    with tempfile.TemporaryDirectory() as td:\n        zf = zipfile.ZipFile(io.BytesIO(file)); zf.extractall(td)\n        AGENT_REGISTRY[agent_id].load_weights(td)\n    return {"status":"ok"}\n''')
marker = '# === ROUTES ==='
idx = code.index(marker) + len(marker)
new_code = code[:idx] + insertion + code[idx:]
src.write_text(new_code)
print('🚀 patched orchestrator with hot-reload endpoint')
PY
}
apply_patch

## 🚀 Launch orchestrator, agents, Ray & mock adapters (background)

In [ ]:
%%bash --bg
cd AGI-Alpha-Agent-v0/alpha_factory_v1
ray start --head --dashboard-host 0.0.0.0 --port 6379 --dashboard-port 8265 &>/dev/null &
uvicorn backend.orchestrator:app --host 0.0.0.0 --port 8000 &>/dev/null &

# tiny PubMed & Carbon adapters
python - <<'PY' &>/dev/null &
from fastapi import FastAPI; import uvicorn, random
app=FastAPI(); @app.get('/')
def root(): return {"msg":"ok"}
uvicorn.run(app,host='0.0.0.0',port=8005)
PY
python - <<'PY' &>/dev/null &
from fastapi import FastAPI; import uvicorn
app=FastAPI(); @app.get('/co2')
def co2(): return {"ppm":420.42}
uvicorn.run(app,host='0.0.0.0',port=8010)
PY

## 🔗 Expose Grafana & Prometheus with `pyngrok`

In [ ]:
from pyngrok import ngrok, conf
conf.get_default().region = 'us'
import os
port = int(os.getenv('DASH_PORT', 9000))
grafana = ngrok.connect(port, 'http'); prom = ngrok.connect(9090, 'http')
print('Grafana  →', grafana.public_url)
print('Prometheus →', prom.public_url)


## 🏋️ Quick k6 load-test (20 VUs × 30 s)

In [ ]:
%%bash
set -euo pipefail
cat > k6.js <<'JS'
import http from 'k6/http'; import {sleep} from 'k6';
export let options = {vus:20,duration:'30s'};
const A=['finance_agent','biotech_agent','climate_agent','manufacturing_agent','policy_agent'];
export default function(){
  const a=A[Math.floor(Math.random()*A.length)];
  http.post(`http://127.0.0.1:8000/agent/${a}/skill_test`,JSON.stringify({ping:Math.random()}),{headers:{'Content-Type':'application/json'}});
  sleep(0.05);
}
JS
k6 run k6.js

## ♻️ Continuous PPO trainer (runs async)

In [ ]:
%%bash --bg
python AGI-Alpha-Agent-v0/alpha_factory_v1/continual/ppo_trainer.py

## ✅ Smoke-probe orchestrator + sample agent call

In [ ]:
import requests, time, pprint, json, os
for _ in range(20):
  try:
    if requests.get('http://127.0.0.1:8000/healthz').status_code==200:
      print('orchestrator healthy'); break
  except: pass; time.sleep(2)
resp=requests.post('http://127.0.0.1:8000/agent/finance_agent/skill_test',json={'ping':123}).json()
pprint.pp(resp)

---
### 🎉 All set!

* Use the **Grafana** link above (`admin / admin`) to explore dashboards (import JSON if blank)
* Rewards tune automatically; edit `rubric.json` in `continual/` and rerun the trainer cell
* Adapt this notebook to plug in your own domain adapters or extra agents
* When finished, run `docker compose down -v` (or `./teardown_alpha_factory_cross_industry_demo.sh`) to clean up

### TroubleshootingThe demo typically completes setup in under 10 minutes. A code cell prints the detected GPU via `nvidia-smi -L` and falls back to the CPU when no GPU is present.Long waits often indicate missing GPU acceleration or network stalls.